In [1]:
pip install -U boto3

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
# SPDX-License-Identifier: Apache-2.0
"""
Shows how to use the Converse API with Anthropic Claude 3 Sonnet (on demand).
"""

import logging
import boto3


from botocore.exceptions import ClientError


logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)


def generate_conversation(bedrock_client,
                    model_id,
                    messages,
                    system_prompts):


    logger.info("Streaming messages with model %s", model_id)
    # Message to send.
    message = {
        "role": "user",
        "content": [{"text": input_text}]
    }
    
    messages = [message]
    system_prompts = [{"text" : system_text}]

    # Inference parameters to use.
    temperature = 0.5
    top_k = 200

    #Base inference parameters to use.
    inference_config = {"temperature": temperature}

    response = bedrock_client.converse_stream(
        modelId=model_id,
        messages=messages,
        system=system_prompts,
        inferenceConfig = inference_config)

    stream = response.get('stream')
    if stream:
        for event in stream:

            if 'messageStart' in event:
                print(f"\nRole: {event['messageStart']['role']}")

            if 'contentBlockDelta' in event:
                print(event['contentBlockDelta']['delta']['text'], end="")

            if 'messageStop' in event:
                print(f"\nStop reason: {event['messageStop']['stopReason']}")

            if 'metadata' in event:
                metadata = event['metadata']
                if 'usage' in metadata:
                    print("\nToken usage")
                    print(f"Input tokens: {metadata['usage']['inputTokens']}")
                    print(
                        f":Output tokens: {metadata['usage']['outputTokens']}")
                    print(f":Total tokens: {metadata['usage']['totalTokens']}")
                if 'metrics' in event['metadata']:
                    print(
                        f"Latency: {metadata['metrics']['latencyMs']} milliseconds")


In [7]:
logging.basicConfig(level=logging.INFO,
                    format="%(levelname)s: %(message)s")

model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
#model_id = "mistral.mistral-large-2402-v1:0"
#model_id = "anthropic.claude-3-haiku-20240307-v1:0"


#system_text = "You are an economist with access to lots of data. Please return the results in Chinese"
system_text = "You are an economist with access to lots of data."
input_text = "Write an article about impact of high inflation to GDP of a country."

try:

    bedrock_client = boto3.client(service_name='bedrock-runtime')

    response = generate_conversation(
        bedrock_client, model_id, system_text, input_text)

except ClientError as err:
    message = err.response['Error']['Message']
    logger.error("A client error occurred: %s", message)
    print(f"A client error occured: {message}")

else:
    print(
        f"Finished generating text with model {model_id}.")



INFO:__main__:Streaming messages with model anthropic.claude-3-sonnet-20240229-v1:0



Role: assistant
Here is a draft article on the impact of high inflation on a country's GDP:

The Damaging Effects of High Inflation on Economic Growth

While moderate inflation is generally viewed as acceptable by economists, high inflation can severely undermine a country's economic performance and growth prospects. When prices rise at an excessive rate, it can create significant headwinds that hold back gross domestic product (GDP) through several key channels.

Erosion of Consumer Purchasing Power
One of the primary drags high inflation has on GDP is that it erodes consumers' real purchasing power. As prices rise faster than wages, households are left with less discretionary income available to spend on goods and services. This reduced consumer spending, which accounts for the majority of GDP in most economies, directly restrains economic growth. Consumers may be forced to rein in purchases of big-ticket items like homes, vehicles, and appliances when inflation is running hot.

Inc